# Imports

## Dependencies

In [3]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from io import BytesIO
import numpy as np
import configparser
import sys

## Micromet

In [4]:
#sys.path.append("G:/Shared drives/UGS_Flux/Data_Processing/Jupyter_Notebooks/MicroMet")
#sys.path.append("C:/Users/kladig/Documents/GitHub/MicroMet")
sys.path.append("../..")
import micromet

## Connection info and Config

In [5]:
config = configparser.ConfigParser()

config.read('../../secrets/config.ini')

from sqlalchemy import create_engine
import urllib.parse
host = config['DEFAULT']['ip']
pw = config['DEFAULT']['pw']
user = config['DEFAULT']['login']

encoded_password = urllib.parse.quote_plus(pw)

def postconn_et(encoded_password, host='localhost',user='postgres',port='5432',db='groundwater', schema = 'groundwater'):
    connection_text = "postgresql+psycopg2://{:}:{:}@{:}:{:}/{:}?gssencmode=disable".format(user,encoded_password,host,port,db)
    return create_engine(connection_text, connect_args={'options': '-csearch_path={}'.format(schema)})


engine = postconn_et(encoded_password, host=host, user=user)

# Pull From Each Datalogger Using the CS Datalogger API

https://help.campbellsci.com/crbasic/cr1000x/Content/Info/webserverapicommands1.htm

In [6]:
site_folders = {#'US-UTD':'Dugout_Ranch',
                #'US-UTB':'BSF',
                #'US-UTJ':'Bluff',
                #'US-UTW':'Wellington',
                #'US-UTE':'Escalante',
                #'US-UTM':'Matheson',
                #'US-UTP':'Phrag',
                'US-CdM':'Cedar_Mesa',
                #'US-UTV':'Desert_View_Myton',
                #'US-UTL':'Pelican_Lake',
                }

#stations = ['UTJ','UTV','UTP','UTW','UTM','CdM','UTL','UTE']
compdf = {}
for stationid,name in site_folders.items():
    station = stationid.split('-')[-1]
    if 'eddy' in config[station].keys():
        ip = config[station]['ip']
        url = f"http://{ip}/tables.html?command=DataQuery&mode=since-record&format=toA5&uri=dl:Flux_AmeriFluxFormat&p1=0"

        response_1 = requests.get(url, auth=HTTPBasicAuth(config['LOGGER']['login'], config['LOGGER']['pw']))
        if response_1.status_code == 200:
            headers = pd.read_csv(BytesIO(response_1.content),skiprows=[0]).iloc[0:2,:].T
            raw_data = pd.read_csv(BytesIO(response_1.content),skiprows=[0,2,3])
            #raw_fold = pathlib.Path('G:/Shared drives/UGS_Flux/Data_Downloads/')
            amflux_out = 'G:/Shared drives/UGS_Flux/Data_Processing/Ameriflux/'

            if raw_data is not None:
                am_data = micromet.Reformatter(raw_data)
            am_df = am_data.et_data
            compdf[stationid] = am_df
            micromet.outfile(am_df, stationid, amflux_out)
        else:
            print(response_1.status_code)
        #if 'met' in config[station].keys():
        print(station)

#url = "http://63.46.183.137:38000/tables.html?command=DataQuery&mode=since-record&format=toA5&uri=dl:Flux_AmeriFluxFormat&p1=0"

403
CdM


In [18]:
#http://192.168.4.14/?command=ClockCheck
station = 'UTJ'
ip = config[station]['ip']
url = f"http://{ip}/?"
#url = f"http://{ip}/tables.html?command=DataQuery&mode=since-record&format=toA5&uri=dl:Flux_AmeriFluxFormat&p1=0"
args = {'command':'ClockCheck',
        'uri':'dl',
        'format':'json',
        }
response_1 = requests.get(url, params=args, auth=HTTPBasicAuth(config['LOGGER']['login'], config['LOGGER']['pw']))

In [19]:
response_1.json()

{'outcome': 1,
 'time': '2025-01-23T07:31:59.94',
 'description': 'The clock was checked'}